# Project: Investigate TMBD Movie Data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
> For this Data Analyst project, I selected the TMDb movie dataset from kaggle to investigate. According to kaggle introduction page, the data contains information that are provided from The Movie Database (TMDb). It collects 5000+ movies and their rating and basic move information, including user ratings and revenue data.


### Metrics for Evaluating the Success Movie
<li>How much REVENUE was generated</li>
<li>The POPULARITY it gained</li>
<li>The average VOTE SCORED</li>

### Potential Key to Affect the Success of a Movie
<li>Budget</li>
<li>Cast</li>
<li>Director</li>
<li>Tagline</li>
<li>Keywords</li>
<li>Runtime</li>
<li>Genres</li>
<li>Production Companies</li>
<li>Release Date</li>
<li>Vote Average</li>

> Since the dataset is featured with the rating of movies as mentioned above, it contains plentiful information for exploring the properties that are associated with successful movies, which can be defined by high popularity, high revenue and high rating score movies. Besides, the dataset also contains the movie released year, so it also can let us to explore the trend in these movie metrics. Therefore, the qestions I am going to explore are including three parts:

### General Characteristics

<li>Question 1: Popularity Over Years</li>
<li>Question 2: The distribution of revenue in different popularity levels in recent five years.</li>
<li>Question 3: The distribution of revenue in different score rating levels in recent five years.</li>

In [1]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.
# Import statements for all of the packages that I plan to use.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html


In [ ]:
# Upgrade pandas to use dataframe.explode() function, which I did. 
!pip install --upgrade pandas==0.25.0

<a id='wrangling'></a>
### Data Wrangling

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
# types and look for instances of missing or possibly errant data.
df = pd.read_csv('tmdb-movies.csv')
df.head()

In [ ]:
# Checking the data info and null values
df.info()

The table above shows that, the dataset contains: 
<li>10866 entries and total</li>
<li>21 columns.</li>
<li>Null values in: Cast, Director, Overview, Genres_columns, Homepage, Tagline, Keywords and Production_companies.</li>
Two columns with too much null values (i.e.Homepage and Tagline) were dropped as they don't add much to asnwering the questions.

In [ ]:
df.describe()

In [ ]:
#to filter the zero budget data
df_budget_zero = df.query('budget == 0')
df_budget_zero.head(2)

In [ ]:
#to filter the zero revenue data
df_revenue_zero = df.query('revenue == 0')
df_revenue_zero.head(2)

In [ ]:
#to count zero values in budget data
df_budget_0count =  df.groupby('budget').count()['id']
df_budget_0count.head(2)

In [ ]:
#to count zero values in revenue data
df_revenue_0count =  df.groupby('revenue').count()['id']
df_revenue_0count.head(2)

In [ ]:
#to count zero values in runtime data
df_runtime_0count =  df.groupby('runtime').count()['id']
df_runtime_0count.head(2)


### Data Cleaning
<li>Columns are dropped that won't add up to answering the research question: homepage, tagline, imdb_id, overview, budget_adj, revenue_adj, runtime.</li>
<li>Columns with duplicate values will be dropped too.</li>
<li>Columns with null values are also dropped: cast, director, and genres</li>
<li>Columns with zero values are replaced with null values: budget and revenue.</li>

In [ ]:
# Dropping the unncessary columns: imdb_id, homepage, tagline, overview, budget_adj, revenue_adj
col = ['imdb_id', 'homepage', 'tagline', 'overview', 'budget_adj', 'revenue_adj']
df.drop(col, axis=1, inplace=True)

In [ ]:
# Checking the dropped columns
df.head(1)

In [ ]:
#Dropping duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#Dropping null values in: cast, director, genres
cal2 = ['cast', 'director', 'genres']
df.dropna(subset = cal2, how='any', inplace=True)
df.isnull().sum()

In [ ]:
#Replacing zero values with null values: budget and revenue
df['budget'] = df['budget'].replace(0, np.NaN)
df['revenue'] = df['revenue'].replace(0, np.NaN)
df.info()

In [ ]:
#Filtering the runtime data with non_zero value
df.query('runtime != 0', inplace=True)
df.query('runtime == 0')

#### Cleaned Dataset

In [ ]:
df.info()
df.describe()

<a id='eda'></a>
## Exploratory Data Analysis


<li>Question 1: Popularity Over Years</li>
<li>Question 2: The distribution of revenue in different popularity levels in recent five years.</li>
<li>Question 3: The distribution of revenue in different score rating levels in recent five years.</li>

<a id='ResearchQuestion'></a>

### QUESTION 1

In [ ]:
df.head(2)

In [ ]:
#Computing the mean for popularity
p_mean = df.groupby('release_year').mean()['popularity']
p_mean.tail()

In [ ]:
# compute the median for popularity
p_median = df.groupby('release_year').median()['popularity']
p_median.tail()

Median values for popularity is more sequenced than for mean.

### QUESTION 2

It will be essential to build a cut_into_quantile function to have the data in four quartile: 'Little', 'Average', 'Much', 'Very Much'

In [ ]:
def cut_into_quantile(dfname ,column_name):
# Quartile maximum and minimum values
    min_value = dfname[column_name].min()
    first_quantile = dfname[column_name].describe()[4]
    second_quantile = dfname[column_name].describe()[5]
    third_quantile = dfname[column_name].describe()[6]
    max_value = dfname[column_name].max()
# Bin_edges to cut quartiles into groups
    bin_edges = [ min_value, first_quantile, second_quantile, third_quantile, max_value]
# The four levels
    bin_names = [ 'Little', 'Average', 'Much', 'Very Much'] 
# Budget_levels column
    name = '{}_range'.format(column_name)
    dfname[name] = pd.cut(dfname[column_name], bin_edges, labels=bin_names, include_lowest = True)
    return dfname

In [ ]:
#Sorting for the most recent 5 years
dfyear =[2011,2012,2013,2014,2015]
df_q2 = pd.DataFrame()

#Applying quartiles to each year
for year in dfyear:
    dfn = df.query('release_year == "%s"' % year)
    dfn2 = cut_into_quantile(dfn,'revenue')
    df_q2 = df_q2.append(dfn2)
df_q2.info()

With the new column created (revenue_levels), let's now explore popularity level in each year

In [ ]:
# Grouping the revenue_levels by year, and calculating the popularity median
dfq2_summary = df_q2.groupby(['release_year','revenue_range']).median()
dfq2_summary.tail(10)

Visualization will portray the findings better ... See below:

In [ ]:
# Setting the positions
pos = list(range(len(dfq2_summary.query('revenue_range =="Little"'))))
width = 0.2 

#The bars and axis
fig, ax = plt.subplots(figsize=(10,5))
plt.bar(pos,
        dfq2_summary.query('revenue_range =="Little"')['popularity'], 
        width,
        alpha=0.5,
        color='#273746',
        label= 'Little') 

plt.bar([p + width for p in pos], 
        dfq2_summary.query('revenue_range =="Average"')['popularity'],
        width, 
        alpha=0.5, 
        color='#B03A2E', 
        label='Average') 

plt.bar([p + width*2 for p in pos], 
        dfq2_summary.query('revenue_range =="Much"')['popularity'], 
        width, 
        alpha=0.5, 
        color='#27AE60', 
        label='Much') 

plt.bar([p + width*3 for p in pos], 
        dfq2_summary.query('revenue_range =="Very Much"')['popularity'], 
        width, 
        alpha=0.5, 
        color='#6C3483', 
        label='Very Much')

ax.set_ylabel('popularity')
ax.set_xlabel('Revenue_Range by Year')
ax.set_title('2015 - 2011 Revenue_Range by Popularity')

ax.set_xticks([p + 1.5 * width for p in pos])
ax.set_xticklabels([2011,2012,2013,2014,2015])

plt.legend( loc='upper left')
plt.grid()
plt.show()

Between 2011 - 2015, movies that have generated very much revenue has increase popularity years.
This depicts that revenue range has a significant relationship with popularity.

### QUESTION 3

To explore the distribution of revenue in different score rating levels (2011 - 2015), almost the same line of code in Question 2 will be applicable here.

In [ ]:
# Grouping the revenue_range by year and finding the vote_average mean
dfq2_summary = df_q2.groupby(['release_year','revenue_range']).mean()
dfq2_summary.tail(10)

Visualizations will make the question more answerable.

In [ ]:
pos = list(range(len(dfq2_summary.query('revenue_range =="Little"'))))
width = 0.2 

#Setting the chart range
fig, ax = plt.subplots(figsize=(12,3))

#Chart parameters
plt.bar(pos,
        dfq2_summary.query('revenue_range =="Little"')['vote_average'],
        width,
        alpha=0.5, 
        color='#DE3163', 
        label= 'Low') 

plt.bar([p + width for p in pos],
        dfq2_summary.query('revenue_range =="Average"')['vote_average'],
        width, 
        alpha=0.5, 
        color='#6495ED', 
        label='Medium') 

plt.bar([p + width*2 for p in pos], 
        dfq2_summary.query('revenue_range =="Much"')['vote_average'], 
        width, 
        alpha=0.5, 
        color='#DFFF00', 
        label='Much') 

plt.bar([p + width*3 for p in pos],
        dfq2_summary.query('revenue_range =="Very Much"')['vote_average'], 
        width, 
        alpha=0.5, 
        color='#1C2833', 
        label='Very Much')

ax.set_ylabel('vote average')
ax.set_xlabel('Revenue Range by Year')
ax.set_title('2011 - 2015 Revenue Range by Vote Average Score')

ax.set_xticks([p + 1.5 * width for p in pos])
ax.set_xticklabels([2015,2014,2013,2012,2011])

plt.ylim(2, 10)

plt.legend(loc='upper center')
plt.grid()
plt.show()

<a id='conclusions'></a>
## Conclusions

<li>QUESTION 1 implies that movie popularity trend increase from 1960 averagely.</li>
<li>QUESTION 2 revealed that, movies with high revenue gained more popularity with 2011 - 2015, and</li>
<li>QUESTION 3 shows the properties associated with increase voting score.</li>

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])